In [1]:
import os 
import time
import random
from packaging import version
import re
import pandas as pd

from tqdm import tqdm

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, accuracy_score

C:\Users\ilkek\anaconda3\envs\pytorchenv\lib\site-packages\sklearn\utils\validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


In [2]:
input_neurons = 9
output_neurons = 4
hidden_neurons= 6
hidden_layer = 1

In [3]:
# setting device as GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [21]:
def init_weights(layer):
    if type(layer) == nn.Linear:
        torch.nn.init.xavier_normal_(layer.weight)
        layer.bias.data.fill_(0.01)
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.build_architecture()
        self.apply(init_weights)

    def build_architecture(self):
        self.encode = nn.Sequential(
            nn.Linear(input_neurons, hidden_neurons),
            nn.Tanh(),
            nn.Linear(hidden_neurons, output_neurons)
        )
        self.decode = nn.Sequential(
            nn.Tanh(),
            nn.Linear( output_neurons, hidden_neurons),
            nn.Tanh(),
            nn.Linear(hidden_neurons, input_neurons),
        )
 
    def forward(self, batch: torch.tensor):
        return self.decode(self.encode(batch))

In [34]:
learning_rate = 0.001
batch_size = 64
num_epochs = 20

In [35]:
# Load Training and Test data
location = os.path.join('', './data/SpotifyFeatures.csv')
  
data = pd.read_csv(location)
data = data.drop_duplicates(subset=['track_id'])
data = data[data['track_id'] != id]

features_used = data[["genre","artist_name","track_name","track_id", "acousticness", "danceability","energy", "instrumentalness", "liveness", "loudness", "speechiness", "tempo", "valence"]]

original_data_all = np.array(features_used.values)

np.random.seed(1)
np.random.shuffle(original_data_all)

original_data1 = original_data_all[:,4:]
print(original_data1.shape)
original_data = np.array(original_data1,"float64")

train_data = original_data[:176771,:]
test_data = original_data[176771:,:]
train_torch = torch.from_numpy(train_data)
test_torch = torch.from_numpy(test_data)

#Dataloader
train_loader = DataLoader(dataset= train_torch, batch_size = batch_size)
test_loader = DataLoader(dataset= test_torch, batch_size = batch_size)

(176774, 9)


In [36]:
#Initialize model
model = Autoencoder().to(device)

In [37]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)

In [38]:
#Train model
for epoch in range(num_epochs):
        epoch_loss = 0.0
        for batch in train_loader:
            batch = batch.to(device=device)
            labels = batch
            optimizer.zero_grad()
            outputs = model(batch.float())
            loss = criterion(outputs, labels.float())
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        epoch_loss = epoch_loss / len(train_loader)
        print(f"Loss of epoch {epoch + 1}: {epoch_loss}")

Loss of epoch 1: 1352.198947747721
Loss of epoch 2: 950.3281499177185
Loss of epoch 3: 642.1350418847409
Loss of epoch 4: 409.8792700227343
Loss of epoch 5: 249.8308811263679
Loss of epoch 6: 156.83438674203305
Loss of epoch 7: 119.70563349963702
Loss of epoch 8: 113.76228884083613
Loss of epoch 9: 113.64307720558479
Loss of epoch 10: 113.64301506730037
Loss of epoch 11: 78.09483003081337
Loss of epoch 12: 28.013788264685683
Loss of epoch 13: 12.735679759782508
Loss of epoch 14: 6.873497602346795
Loss of epoch 15: 5.283136257174392
Loss of epoch 16: 4.767644551626116
Loss of epoch 17: 4.563745536926731
Loss of epoch 18: 4.471057010741067
Loss of epoch 19: 4.417368678178556
Loss of epoch 20: 4.379208972908818


In [39]:
def get_features(name):
    def hook(model, input, output):
        features[name] = output.detach()
    return hook